In [ ]:
import sagemaker
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
import os

In [ ]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
local_source_code_dir = './'

In [ ]:
entry_point_script = 'main.py'

In [ ]:
input_data_s3_path = f"s3://{sagemaker_session.default_bucket()}/stock-data-for-portfolio-optimization/stock_daily_returns.csv"
output_data_s3_path = f"s3://{sagemaker_session.default_bucket()}/optimized_portfolios/"

In [ ]:
processor = ScriptProcessor(
    image_uri=sagemaker.image_uris.retrieve(
        sagemaker_session.boto_region_name, "python", "3.9"
    ),
    role=role,
    instance_count=1,
    instance_type='ml.t3.medium',
    command=['python3'],
    source_dir=local_source_code_dir,
    entry_point=entry_point_script,
)

In [ ]:
processor.run(
    inputs=[
        ProcessingInput(
            source=input_data_s3_path,
            destination='/opt/ml/processing/input/data',
            s3_data_type='S3Prefix',
            s3_input_mode='File'
        )
    ],
    outputs=[
        ProcessingOutput(
            source='/opt/ml/processing/output/results',
            destination=output_data_s3_path
        )
    ],
    arguments=[
        '--input_data_path', os.path.join('/opt/ml/processing/input/data', 'stock_daily_returns.csv'),
        '--output_data_path', '/opt/ml/processing/output/results',
        '--num_generations', '200',
        '--population_size', '100',
        '--risk_free_rate', '0.025',
        '--crossover_rate', '0.7',
        '--mutation_rate', '0.15',
        '--elitism_count', '5'
    ]
)

In [ ]:
print(f"SageMaker Processing Job launched. Check CloudWatch logs for progress.")
print(f"Results will be in S3: {output_data_s3_path}")